In [7]:
!pip install -q duckdb dbt-duckdb ducklake-delta-exporter  --upgrade
import sys
sys.exit(0)

sys.exit called with value 0. The interpreter will be restarted.


In [8]:
import os
os.environ['DBT_SCHEMA']              = 'power'  
os.environ['FABRIC_WORKSPACE']        = 'dbt'
os.environ['FABRIC_LAKEHOUSE']        = 'data'
os.environ['DUCKLAKE_METADATA_PATH']  = 'sqlite:/lakehouse/default/Files/metadata.db' # you need to attach a lakehouse, DB is read only on abfss unfortunately :(
os.environ['download_limit']          = '100'

In [9]:
!git clone https://github.com/djouallah/dbt.git /tmp/dbt
!cd /tmp/dbt && dbt run && dbt test

fatal: destination path '/tmp/dbt' already exists and is not an empty directory.
12:38:15  Running with dbt=1.11.2
12:38:16  Registered adapter: duckdb=1.10.0
12:38:16  Found 8 models, 16 data tests, 1 source, 476 macros
12:38:16  
12:38:16  Concurrency: 1 threads (target='dev')
12:38:16  
12:38:18  1 of 8 START sql incremental model power.dim_calendar .......................... [RUN]
12:38:18  1 of 8 OK created sql incremental model power.dim_calendar ..................... [OK in 0.46s]
12:38:18  2 of 8 START sql view model power_staging.stg_csv_archive_log .................. [RUN]
12:38:18  [DOWNLOAD] Starting download with PATH_ROOT=abfss://dbt@onelake.dfs.fabric.microsoft.com/data.Lakehouse, csv_archive_path=abfss://dbt@onelake.dfs.fabric.microsoft.com/data.Lakehouse/Files/csv, download_limit=100
12:38:18  [DOWNLOAD] Daily source: aemo
12:38:19  [DOWNLOAD] Processing daily files...
12:38:21  [DOWNLOAD] Processing intraday SCADA files...
12:39:38  [DOWNLOAD] Processing intraday PRIC

In [10]:
from ducklake_delta_exporter import generate_latest_delta_log
generate_latest_delta_log('/lakehouse/default/Files/metadata.db')

  needs_export: 8 tables

📦 Exporting 8 tables...

  Processing power.fct_summary...
  ✅ power.fct_summary: exported snapshot 15 (2 files)

  Processing power.fct_scada_today...
  ✅ power.fct_scada_today: exported snapshot 14 (2 files)
  ⏭️  power.fct_scada: snapshot 8 already exported

  Processing power.fct_price_today...
  ✅ power.fct_price_today: exported snapshot 13 (2 files)
  ⏭️  power.fct_price: snapshot 6 already exported
  ⏭️  power.dim_duid: snapshot 5 already exported

  Processing power.csv_archive_log...
  ✅ power.csv_archive_log: exported snapshot 11 (6 files)
  ⏭️  power.dim_calendar: snapshot 3 already exported

🎉 Export completed!


In [11]:
import duckdb
duckdb.sql(f"""select max(cutoff), count(*) from delta_scan
('abfss://{os.environ['FABRIC_WORKSPACE']}@onelake.dfs.fabric.microsoft.com/{os.environ['FABRIC_LAKEHOUSE']}.Lakehouse/Tables/{os.environ['DBT_SCHEMA']}/fct_summary') 
 """).show()

┌──────────────────────────┬──────────────┐
│       max(cutoff)        │ count_star() │
│ timestamp with time zone │    int64     │
├──────────────────────────┼──────────────┤
│ 2026-02-02 22:40:00+00   │      3923004 │
└──────────────────────────┴──────────────┘

